## Train BERT model

In this notebook we will train a BERT model for Sentiment Analysis in English

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
from datasets import Dataset, Value, ClassLabel, Features
from pysentimiento.preprocessing import preprocess_tweet
from pysentimiento.semeval import load_datasets


train_dataset, dev_dataset, test_dataset = load_datasets()


39716 9929 20632


In [4]:
%load_ext autoreload
%autoreload 2
import os
from pysentimiento.tass import load_model
from pysentimiento.emotion.datasets import id2label, label2id

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

base_model = "bert-base-uncased"

model, tokenizer = load_model(base_model, 
    id2label=id2label, 
    label2id=label2id
)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading model bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

batch_size = 32

eval_batch_size = 16

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)


In [6]:
def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['label']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)

In [7]:
from transformers import TrainingArguments, Trainer
from pysentimiento.emotion.metrics import compute_metrics

epochs = 10

total_steps = (epochs * len(train_dataset)) // batch_size
warmup_steps = total_steps // 10
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    warmup_steps=warmup_steps,
    evaluation_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.681900,0.711082,0.679927,0.653643,0.706350,0.644163
2,0.560200,0.637785,0.728774,0.702815,0.732001,0.686996
3,0.328300,0.775216,0.731796,0.708863,0.730011,0.695627
4,0.172600,0.977231,0.722530,0.712389,0.706290,0.720811
5,0.109000,1.441598,0.707725,0.674794,0.720382,0.657420
6,0.069000,1.504206,0.716789,0.699764,0.714652,0.689349
7,0.050200,1.754003,0.716789,0.698738,0.710299,0.690480
8,0.033900,1.918710,0.722329,0.707749,0.711089,0.704683
9,0.017900,1.991970,0.722429,0.704502,0.715994,0.695650
10,0.011700,2.100404,0.723638,0.706298,0.715355,0.699052


TrainOutput(global_step=12420, training_loss=0.21754982490662408, metrics={'train_runtime': 5652.3402, 'train_samples_per_second': 2.197, 'total_flos': 0, 'epoch': 10.0})

In [5]:
from pysentimiento import compute_metrics

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.781631,0.653295,0.641696,0.642772,0.650757


TrainOutput(global_step=151, training_loss=0.8435698250271627)

In [7]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.7347931861877441,
 'eval_accuracy': 0.686398678414097,
 'eval_f1': 0.6673146659667489,
 'eval_precision': 0.6676487102833413,
 'eval_recall': 0.6695717138336971,
 'epoch': 1.0}

In [ ]:
path = "../models/beto-sentiment-analysis"
model.save_pretrained(path)
tokenizer.save_pretrained(path)